In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU devices found.")

physical_devices = tf.config.list_physical_devices('GPU')
print(tf.config.list_physical_devices('GPU'))



No GPU devices found.
[]


In [17]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype("float32")/255.0
x_test  = x_test.reshape(-1, 28*28).astype("float32")/255.0

### Here we make custom model. 

In [23]:
########    Making layers by my self
class Dense(layers.Layer):
    #def __init__(self, units, input_dim):
    #    super(Dense, self).__init__()
    #    self.w = self.add_weight(
    #        name = 'w',  # without name we cant save a model
    #        shape=(input_dim, units),
    #        initializer = 'random_normal',
    #        trainable = True,
    #    )

    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(
            name = 'w',  # without name we cant save a model
            shape=(input_shape[-1], self.units),
            initializer = 'random_normal',
            trainable = True,
        ) 
        self.b = self.add_weight(
            name='b', shape=(self.units, ), initializer = 'zeros',
            trainable = True,
        )
        
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
        

In [26]:
class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()
        
    def call(self, x):
        return tf.math.maximum(x, 0)
    
# tf.math is already existing funciton

In [28]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu=MyReLU()    # this is used in line number 11
        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)
        
    def call(self, input_tensor):
        x=self.relu(self.dense1(input_tensor))
        return self.dense2(x)
    
model=MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 6s - loss: 0.3372 - 6s/epoch - 3ms/step
Epoch 2/2
1875/1875 - 6s - loss: 0.1582 - 6s/epoch - 3ms/step
313/313 - 1s - loss: 0.1377 - 966ms/epoch - 3ms/step


0.13767801225185394